### 금융상품 갱신 여부 예측하는 ANN

Churn_Modelling.csv 파일을 보면, 고객 정보와 해당 고객이 금융상품을 갱신했는지 안했는지의 여부에 대한 데이터가 있다.

이 데이터를 가지고 갱신여부를 예측하는 딥러닝을 구성하시오.

# Neural Networks and Deep Learning

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

In [2]:
# 1. csv 파일을, 구글이 제공한 컴퓨터에 업로드 하여 사용하는 방법

In [3]:
df = pd.read_csv('Churn_Modelling.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Churn_Modelling.csv'

In [ ]:
# 2. 내 구글 드라이브에 csv파일을 갖다놓고, 코랩을 연결시키는 방법

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML2/data/Churn_Modelling.csv')

In [6]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [9]:
y = df[ 'Exited']

In [10]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [11]:
X = df.loc[ : , 'CreditScore' :  'EstimatedSalary' ]

In [12]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [13]:
# prompt: Geography 컬럼은 몇개의 데이터로 구성되어있나

unique_geography = df['Geography'].unique()
print(len(unique_geography))


3


In [14]:
# prompt: Geography 컬럼에 nunique 해줘

df['Geography'].nunique()



3

In [15]:
X['Gender'].nunique()

2

In [16]:
# prompt: gender 컬럼을 레이블 인코딩 해줘

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])
X.head(2)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58


In [17]:
sorted( df['Geography'].unique() )

['France', 'Germany', 'Spain']

In [18]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [19]:
# prompt: geograpy 컬럼을 columtransformer 로 원핫 인코딩 해줘

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    [('encoding', OneHotEncoder(), [1])],   # The column numbers to be encoded
    remainder='passthrough'                         # Leave other columns as is
)

X = ct.fit_transform(X)


In [20]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [21]:
# Dummy Variable Trap 을 적용하자.

# 딥러닝에서는 컬럼하나가 연산에 아주 크게 작용하므로,

# 원핫 인코딩한 결과에서, 가장 왼쪽의 컬럼은 삭제해도
# 데이터를 표현하는 데는 아무 문제 없다.

# France, Germerny, Spain 3개 컬럼으로 원핫 인코딩 되는데
#    1        0       0
#    0        1       0
#    0        0       1

# 맨 왼쪽 France 컬럼을 삭제해도,
#  0   0   => France
#  1   0   => Germeny
#  0   1   => Spain

In [22]:
pd.DataFrame(X).drop(0, axis=1).values

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [23]:
# prompt: X 의 첫번째 컬럼을 삭제해줘

X = pd.DataFrame(X).drop(0, axis=1).values


In [24]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [25]:
# prompt: X 를 스탠다드 스케일러로 피처 스케일링 해줘

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [26]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [28]:
# Part 2 - Now let's make the ANN!

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [29]:
X.shape

(10000, 11)

In [30]:
X[ 0 , ]

array([-0.57873591, -0.57380915, -0.32622142, -1.09598752,  0.29351742,
       -1.04175968, -1.22584767, -0.91158349,  0.64609167,  0.97024255,
        0.02188649])

In [31]:
# 깡통 인공지능
model = Sequential()

In [32]:
model.add(  Dense(units = 8, activation='relu', input_shape= (11,)  )   )

In [33]:
model.add(  Dense(6, 'relu')  )

In [34]:
model.add(  Dense(1, 'sigmoid') )

In [49]:
12*8

96

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 96        
                                                                 
 dense_1 (Dense)             (None, 6)                 54        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 157 (628.00 Byte)
Trainable params: 157 (628.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# 모델링이 끝나면, 컴파일(Compile)을 해야 한다.
# 컴파일이란,
#     옵티마이저(optimizer) 셋팅
#     로스펑션(loss function 오차함수,손실함수) 셋팅
#     검증(평가)방법 셋팅

In [37]:

# 2개로 분류하는 문제의 loss는 'binary_crossentropy' 로 설정한다.

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'] )

In [38]:
# 컴파일이 끝나면, 학습을 한다.

In [39]:
model.fit( X_train, y_train, batch_size = 10, epochs = 20  )

Epoch 1/20
800/800 [==============================] - 2s 2ms/step - loss: 0.6598 - accuracy: 0.6844
Epoch 2/20
800/800 [==============================] - 1s 2ms/step - loss: 0.5221 - accuracy: 0.8056
Epoch 3/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4676 - accuracy: 0.8164
Epoch 4/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4305 - accuracy: 0.8296
Epoch 5/20
800/800 [==============================] - 1s 2ms/step - loss: 0.4013 - accuracy: 0.8405
Epoch 6/20
800/800 [==============================] - 1s 2ms/step - loss: 0.3820 - accuracy: 0.8481
Epoch 7/20
800/800 [==============================] - 2s 2ms/step - loss: 0.3700 - accuracy: 0.8515
Epoch 8/20
800/800 [==============================] - 2s 2ms/step - loss: 0.3635 - accuracy: 0.8510
Epoch 9/20
800/800 [==============================] - 1s 2ms/step - loss: 0.3583 - accuracy: 0.8516
Epoch 10/20
800/800 [==============================] - 1s 2ms/step - loss: 0.3551 - accuracy: 0.8556

In [40]:
# 학습이 끝나면, 평가를한다. => X_test

In [41]:
model.evaluate( X_test, y_test )

63/63 [==============================] - 0s 1ms/step - loss: 0.3357 - accuracy: 0.8625


[0.33572834730148315, 0.862500011920929]

In [42]:
# 컨퓨전 매트릭스로 확인!!

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [46]:
y_pred = model.predict(X_test)

63/63 [==============================] - 0s 4ms/step


In [47]:
y_pred

array([[0.20139079],
       [0.26640037],
       [0.11788324],
       ...,
       [0.15886998],
       [0.16503814],
       [0.19359547]], dtype=float32)

In [53]:
y_pred = (y_pred > 0.5).astype(int)

In [54]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [48]:
y_test

9394    0
898     1
2398    0
5906    0
2343    0
       ..
1037    0
2899    0
9549    0
2740    0
6690    0
Name: Exited, Length: 2000, dtype: int64

In [56]:
cm = confusion_matrix(y_test, y_pred)

In [57]:
cm

array([[1518,   77],
       [ 198,  207]])

In [59]:
(1518+207) / cm.sum()

0.8625

In [60]:
accuracy_score(y_test, y_pred)

0.8625

In [61]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 3ms/step - loss: 0.3357 - accuracy: 0.8625


[0.33572834730148315, 0.862500011920929]

In [62]:
model.layers

In [77]:
model.layers[0].get_weights()

[array([[ 0.0255513 ,  0.39790332,  0.06775836,  0.00450788,  0.46269757,
         -0.07763985, -0.01292914,  0.303494  ],
        [ 0.13346708, -0.15089357, -0.15101895, -0.13871048,  0.09091842,
         -0.0833789 , -0.4647423 , -0.69497806],
        [ 0.04009793, -0.1292692 ,  0.05017538,  0.03303674, -0.09677085,
         -0.07772572,  0.0307756 , -0.02130013],
        [-0.05977811, -0.22613963, -0.04058979, -0.02640794, -0.127124  ,
          0.22795096,  0.20796376, -0.27261108],
        [ 0.31088653, -0.2525451 , -1.2578822 , -0.10339174,  0.0434383 ,
         -0.35552147, -0.05514986, -0.28676036],
        [ 0.03412293, -0.06408243, -0.21445014, -0.10064012, -0.05698439,
          0.4639877 , -0.4139054 , -0.09537812],
        [ 0.1518224 ,  0.30439103, -0.10671306, -0.6307266 , -0.01284001,
         -0.43516272,  0.23779579,  0.802353  ],
        [ 0.44210285,  1.2385817 , -0.0882901 , -1.1235752 , -0.79400986,
          0.15540053, -0.7941552 , -0.22976528],
        [-0.0281

In [80]:
# 인풋레이어에서 히든레이어로 연결하는 선 (방정식의 계수)
#  11 * 8
model.layers[0].get_weights()[0]

array([[ 0.0255513 ,  0.39790332,  0.06775836,  0.00450788,  0.46269757,
        -0.07763985, -0.01292914,  0.303494  ],
       [ 0.13346708, -0.15089357, -0.15101895, -0.13871048,  0.09091842,
        -0.0833789 , -0.4647423 , -0.69497806],
       [ 0.04009793, -0.1292692 ,  0.05017538,  0.03303674, -0.09677085,
        -0.07772572,  0.0307756 , -0.02130013],
       [-0.05977811, -0.22613963, -0.04058979, -0.02640794, -0.127124  ,
         0.22795096,  0.20796376, -0.27261108],
       [ 0.31088653, -0.2525451 , -1.2578822 , -0.10339174,  0.0434383 ,
        -0.35552147, -0.05514986, -0.28676036],
       [ 0.03412293, -0.06408243, -0.21445014, -0.10064012, -0.05698439,
         0.4639877 , -0.4139054 , -0.09537812],
       [ 0.1518224 ,  0.30439103, -0.10671306, -0.6307266 , -0.01284001,
        -0.43516272,  0.23779579,  0.802353  ],
       [ 0.44210285,  1.2385817 , -0.0882901 , -1.1235752 , -0.79400986,
         0.15540053, -0.7941552 , -0.22976528],
       [-0.02816006, -0.04886988

In [79]:
# 상수항과 히든레이어와 연결된 선 ( 상수항 )
model.layers[0].get_weights()[1]

array([-0.21410765,  0.25200865,  0.95649266, -0.02172018,  0.43890074,
        0.25057012, -0.24783555, -0.03401265], dtype=float32)

In [83]:
model.layers[1].get_weights()[0]

array([[-0.6431609 ,  0.62959874,  0.42446238,  0.47494084,  0.29682535,
         0.27023178],
       [-0.04152434,  0.763938  , -0.745082  , -0.68375903,  0.7473379 ,
         0.19858594],
       [-0.6947217 , -0.5636261 , -0.35327655, -0.5381766 , -0.47831345,
        -1.2258986 ],
       [-0.4184302 ,  0.55816334,  0.46585336,  0.9682358 , -0.1118132 ,
        -0.2739642 ],
       [-0.49937078, -0.32520527,  0.6655418 ,  0.74609315, -0.86297333,
         0.04011969],
       [-0.5479509 , -0.40425918,  0.0068292 , -0.09999155, -0.28857613,
        -0.9936878 ],
       [ 0.40421066,  0.6254157 , -0.20345904, -0.23183449,  0.42591733,
         0.7595272 ],
       [-0.34625468, -0.462322  ,  0.43266702,  0.17152178, -0.85329396,
         0.35673097]], dtype=float32)

In [84]:
model.layers[1].get_weights()[1]

array([-0.175802  , -0.03929821, -0.13649046, -0.09058762, -0.41193867,
        0.1188187 ], dtype=float32)

In [86]:
model.layers[2].get_weights()

[array([[0.25386566],
        [1.0793781 ],
        [0.8603833 ],
        [0.7250965 ],
        [1.6191521 ],
        [0.71235734]], dtype=float32),
 array([-3.0248134], dtype=float32)]

## 다음 신규 데이터를 통해 분류해 보자

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40
- Tenure: 3
- Balance: 60000
- Number of Products: 2
- Has Credit Card: Yes
- Is Active Member: Yes
- Estimated Salary: 50000

In [88]:
df.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [178]:
new_data = [[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000 ]]

In [179]:
df_new_data = pd.DataFrame(new_data)

In [180]:
df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1 non-null      int64 
 1   1       1 non-null      object
 2   2       1 non-null      object
 3   3       1 non-null      int64 
 4   4       1 non-null      int64 
 5   5       1 non-null      int64 
 6   6       1 non-null      int64 
 7   7       1 non-null      int64 
 8   8       1 non-null      int64 
 9   9       1 non-null      int64 
dtypes: int64(8), object(2)
memory usage: 208.0+ bytes


In [181]:
df_new_data

,0,1,2,3,4,5,6,7,8,9
0,600,France,Male,40,3,60000,2,1,1,50000


In [182]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [183]:
df_new_data.columns = ['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

In [184]:
df_new_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [185]:
df_new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CreditScore      1 non-null      int64 
 1   Geography        1 non-null      object
 2   Gender           1 non-null      object
 3   Age              1 non-null      int64 
 4   Tenure           1 non-null      int64 
 5   Balance          1 non-null      int64 
 6   NumOfProducts    1 non-null      int64 
 7   HasCrCard        1 non-null      int64 
 8   IsActiveMember   1 non-null      int64 
 9   EstimatedSalary  1 non-null      int64 
dtypes: int64(8), object(2)
memory usage: 208.0+ bytes


In [186]:
df_new_data['Gender'] = label_encoder.transform(df_new_data['Gender'])

In [187]:
df_new_data = ct.transform(df_new_data)

In [188]:
new_data = pd.DataFrame(df_new_data).drop(0, axis =1).values

In [189]:
new_data = sc.transform(new_data)

In [191]:
y_pred = model.predict(new_data)

1/1 [==============================] - 0s 31ms/step


In [194]:
(y_pred > 0.5).astype(int)

array([[0]])

In [160]:
new_data2 = [{'CreditScore': 600, 'Geography': 'France', 'Gender': 'Male', 'Age':40, 'Tenure': 3, 'Balance': 60000, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary':50000 }]

In [162]:
df_new_data2 = pd.DataFrame(new_data2)

In [165]:
df_new_data2['Gender'] = label_encoder.transform(df_new_data2['Gender'])

In [167]:
df_new_data2 = ct.transform(df_new_data2)

In [169]:
new_data2 = pd.DataFrame(df_new_data2).drop(0, axis =1).values

In [171]:
new_data2 = sc.transform(new_data2)

In [172]:
new_data2

array([[-0.57873591, -0.57380915, -0.52281016,  0.91241915,  0.10281024,
        -0.69598177, -0.26422114,  0.80773656,  0.64609167,  0.97024255,
        -0.87101922]])

In [174]:
y_pred = model.predict(new_data2)

1/1 [==============================] - 0s 31ms/step


In [177]:
(y_pred > 0.5).astype(int)

array([[0]])

### 용어 정리

epoch

- 한 번의 epoch는 신경망에서 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것을 말함. 즉, 전체 데이터 셋에 대해 한 번 학습을 완료한 상태


batch_size

메모리의 한계와 속도 저하 때문에 대부분의 경우에는 한 번의 epoch에서 모든 데이터를 한꺼번에 집어넣을 수는 없습니다. 그래서 데이터를 나누어서 주게 되는데 이때 몇 번 나누어서 주는가를 iteration, 각 iteration마다 주는 데이터 사이즈를 batch size라고 합니다.

출처: https://www.slideshare.net/w0ong/ss-82372826

### GridSearch 를 이용한, 최적의 하이퍼 파라미터 찾기

In [43]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense


ModuleNotFoundError: No module named 'keras.wrappers'